# Extract thông tin tài chính từ tin tức.

Để đánh giá tình hình tài chính của một công ty, một trong những nguồn thông tin quan trọng là các bài báo, tin tức về công ty đó. Tuy nhiên, việc đọc và tổng hợp thông tin từ các bài báo là một công việc tốn thời gian và công sức. Trong bài toán này, chúng ta sẽ xây dựng một mô hình để tự động trích xuất thông tin tài chính từ các bài báo về công ty.

```
{công ty A} => {có_lợi_nhuận, không_có_lợi_nhuận, lợi_nhuận_tăng, lợi_nhuận_giảm, v.v.} => {tiền_tệ, số_tiền, v.v.}
```

https://neo4j.com/labs/genai-ecosystem/llamaindex/


In [1]:
from dotenv import load_dotenv
import os
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

import nest_asyncio
nest_asyncio.apply()
# Install the python-dotenv package if not already installed
%pip install python-dotenv

# Load the .env file

load_dotenv('../.env')

# Access the OpenAI key
openai_key = os.getenv("OPENAI_API_KEY")

llm = OpenAI(model="gpt-4o-mini", api_key=openai_key)
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embed_model

FOLDER_DATA = "../data/bao-chi"

Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

documents = SimpleDirectoryReader(FOLDER_DATA).load_data()
print("length of documents:", len(documents))
print('documents:', documents[1])



length of documents: 24
documents: Doc ID: dba4f0a5-fe01-47e5-a0d4-3b71470e0401
Text: Khối ngoại bất ngờ có tuần bán ròng hơn 2.000 tỷ, cổ phiếu “đại
gia” công nghệ số 1 Việt Nam dẫn đầu danh sách xả hàng


In [17]:
graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="ttt@123ASD",
    url="bolt://localhost:7687",
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [4]:
from llama_index.core import StorageContext, load_index_from_storage

index = load_index_from_storage(
    StorageContext.from_defaults(persist_dir="./storage")
)

In [5]:

# Define retriever
retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)

question = "CTCP Trung tâm Hội chợ Triển lãm Việt Nam (VEFAC – mã VEF) có lợi nhuận gộp bao nhiêu ?"
results = retriever.retrieve(question)
print("Retrieved nodes:")
for record in results:
    print("record:", record.text)

print("Ask:")
# Question answering
query_engine = index.as_query_engine(include_text=True)
response = query_engine.query(question)
print(str(response))

Retrieved nodes:
Ask:
Empty Response


In [41]:
# Define custom retriever-
from pydantic import BaseModel
from typing import Optional, List


class Entities(BaseModel):
    """Liệt kê tất cả [công ty, mã chứng khoán, tổ chức tài chính, Khối ngoại, Tự doanh] và các thực thể khác trong văn bản sau"""
    names: Optional[List[str]]


prompt_template_entities = """
Liệt kê tất cả tên [công ty, mã chứng khoán, tổ chức tài chính, Khối ngoại, Tự doanh] và các thực thể khác trong văn bản sau:
{text}
"""

from typing import Any, Optional

from llama_index.core.embeddings import BaseEmbedding
from llama_index.core.retrievers import CustomPGRetriever, VectorContextRetriever
from llama_index.core.vector_stores.types import VectorStore
from llama_index.program.openai import OpenAIPydanticProgram


class MyCustomRetriever(CustomPGRetriever):
    """Custom retriever with cohere reranking."""

    def init(
        self,
        ## vector context retriever params
        embed_model: Optional[BaseEmbedding] = None,
        vector_store: Optional[VectorStore] = None,
        similarity_top_k: int = 4,
        path_depth: int = 3,
        include_text: bool = True,
        **kwargs: Any,

    ) -> None:
        """Uses any kwargs passed in from class constructor."""
        self.entity_extraction = OpenAIPydanticProgram.from_defaults(
            output_cls=Entities, prompt_template_str=prompt_template_entities
        )
        self.vector_retriever = VectorContextRetriever(
            graph_store,
            include_text=self.include_text,
            embed_model=embed_model,
            similarity_top_k=similarity_top_k,
            path_depth=path_depth,
        )
    def custom_retrieve(self, query_str: str) -> str:
        """Define custom retriever with reranking.

        Could return `str`, `TextNode`, `NodeWithScore`, or a list of those.
        """
        entities = self.entity_extraction(text=query_str).names
        result_nodes = []
        if entities:
            print(f"Detected entities: {entities}")
            for entity in entities:
                result_nodes.extend(self.vector_retriever.retrieve(entity))
        else:
            result_nodes.extend(self.vector_retriever.retrieve(query_str))
        ## TMP: please change
        final_text = "\n\n".join(
            [n.get_content(metadata_mode="llm") for n in result_nodes]
        )
        return final_text


In [42]:
from llama_index.core.query_engine import RetrieverQueryEngine

custom_sub_retriever = MyCustomRetriever(
    graph_store,
    include_text=True,
    vector_store=index.vector_store,
    embed_model=embed_model,
)


query_engine = RetrieverQueryEngine.from_args(
    index.as_retriever(sub_retrievers=[custom_sub_retriever]), llm=llm
)

In [43]:
test_pairs = [
    ("lãi sau thuế của Chứng khoán LPBank ?", "Tăng 275% so với cùng kỳ năm trước"),
    ("thu gom cố phiếu MSN?", "Khối ngoại cũng giải ngân mua ròng 179 tỷ tại cổ phiếu MSN"),
    ("Doanh thu Bia Sài Gòn - Quảng Ngãi (BSQ) trong BCTC quý 3/2024 ?", "Bia Sài Gòn - Quảng Ngãi (BSQ) ghi nhận doanh thu 418 tỷ đồng"),
    ("Lực mua ròng các cổ phiếu nào ?", "ACV', 'YEG', 'TCB', 'TPB', 'MWG'"),
    ("VEFAC là công ty con của ?", "VIC")
]

for question, expected_answer in test_pairs:
    response = query_engine.query(question)
    nodes = retriever.retrieve(question)
    print("Retrieved nodes:")
    for node in nodes:
        print(f"Node: {node.text}")
    print(f"Question: {question}")
    print(f"Expected Answer: {expected_answer}")
    print(f"Actual Answer: {response.response}")
    print()

Detected entities: ['Chứng khoán LPBank']
Retrieved nodes:
Question: lãi sau thuế của Chứng khoán LPBank ?
Expected Answer: Tăng 275% so với cùng kỳ năm trước
Actual Answer: Lãi sau thuế của Chứng khoán LPBank đạt 33 tỷ đồng, tăng 275% so với cùng kỳ năm trước.

Detected entities: ['MSN']
Retrieved nodes:
Question: thu gom cố phiếu MSN?
Expected Answer: Khối ngoại cũng giải ngân mua ròng 179 tỷ tại cổ phiếu MSN
Actual Answer: Thông tin về việc thu gom cổ phiếu MSN không được đề cập trong nội dung đã cung cấp. Nếu bạn cần thông tin cụ thể về cổ phiếu này, có thể tham khảo các nguồn tài chính hoặc báo cáo thị trường chứng khoán.

Detected entities: ['Bia Sài Gòn - Quảng Ngãi', 'BSQ']
Retrieved nodes:
Question: Doanh thu Bia Sài Gòn - Quảng Ngãi (BSQ) trong BCTC quý 3/2024 ?
Expected Answer: Bia Sài Gòn - Quảng Ngãi (BSQ) ghi nhận doanh thu 418 tỷ đồng
Actual Answer: Doanh thu của Bia Sài Gòn - Quảng Ngãi (BSQ) trong BCTC quý 3/2024 là 418 tỷ đồng.

Detected entities: ['Lực mua ròng', 'cổ

## Kết luận.

Nhiều sai sót và thật nhiều ảo giác, số 1000 tỷ đồng ở mô ra rứa trời =))
"VEFAC là công ty con của ?" : câu này dễ mà cũng không trả lời được. 